In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import datetime as dt
import math
import os
from itertools import product
from datetime import datetime
from dateutil.relativedelta import relativedelta
from quant_functions import anal_funcs, data_funcs

import yfinance as yf
## FRED 정보 수집
from fredapi import Fred

In [2]:
# 시각화 관련
import plotly_express as px
import plotly.figure_factory as ff
import cufflinks as cf
cf.go_offline(connected=True)
## cf.getThemes()
cf.set_config_file(theme='polar')
import plotly.graph_objects as go

from matplotlib.colors import LinearSegmentedColormap
from matplotlib.ticker import FuncFormatter
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set_style('whitegrid')
## plt.style.use("fivethirtyeight")
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams["figure.figsize"] = (10,6)

In [3]:
## 그리기 설정값
color_basic = "#3b4a6b" # 네이비 기본색
color_norm = "#f23557" # 진홍색 강조색
color_cont = "#22b2da" # 파란색 대비색
color_sub = "#ff6f3c" # 주황색 보조색
color_around = "#6E6E6E" # 회색 주변색
color_around2 = "#E0E0E0" # 연회색 주변색

figsize=(12, 8)

##### Function #####

In [ ]:
def simulate_portfolio_assets(asset_rets_df, weights_list, initial_investment=100):
    """
    n개의 자산에 대한 투자 비중에 따른 포트폴리오 시뮬레이션.

    Parameters:
        asset_returns (list of pd.Series): 각 자산의 수익률 시계열 데이터프레임.
        weights_list (list of list): 자산별 비중 리스트. 각 리스트의 합은 1이어야 함.
        initial_investment (float): 초기 투자 금액 (기본값: 100).

    Returns:
        pd.DataFrame: 각 비중 조합에 대한 포트폴리오 가치 시계열 데이터프레임.
    """
    asset_rets_list = [asset_rets_df[col] for col in asset_rets_df.columns]
    portfolio_values = {}
    for weights in weights_list:
        _weights = [i/sum(weights) for i in weights]
        # 포트폴리오 수익률 계산 (각 자산의 비중 곱)
        portfolio_returns = sum(w * r for w, r in zip(_weights, asset_rets_list))
        # 초기 투자 금액 기준 포트폴리오 가치 시계열 계산
        portfolio_cum_value = initial_investment * (1 + portfolio_returns).cumprod()
        portfolio_values[str(weights)] = portfolio_cum_value

    # 데이터프레임으로 변환
    portfolio_df = pd.DataFrame(portfolio_values)
    return portfolio_df

def split_time_series(df, n_days):
    split_days = n_days
    dfs = []
    for i in range(len(df)-split_days+1):
        dfs.append(df[i:split_days+i])
    return dfs

## beta를 이용한 imputation
def imputation(df, basis_name, target_name):
    dataset = df.copy()
    rets = dataset[[target_name, basis_name]].dropna(subset=[basis_name]).pct_change()
    
    # 공분산과 분산 계산
    cov_matrix = np.cov(rets.dropna()[target_name], rets.dropna()[basis_name])
    cov_stock_market = cov_matrix[0, 1]  # 공분산
    var_market = cov_matrix[1, 1]        # market 분산
    # 베타 계산
    beta = cov_stock_market / var_market
    rets[target_name][rets[target_name].isna()] = rets[basis_name]*beta
    rets = rets.fillna(0)
    df_imputed = (rets+1).cumprod()
    dataset[target_name] = df_imputed[target_name]
    print(beta)
    return dataset

##### Data Load #####

In [38]:
index = ['최저1인', '최저2인', '최저3인', '최저4인', '중위1인', '중위2인', '중위3인', '중위4인', '상위']
need_cash_ls = [134, 221, 283, 344, 224, 368, 471, 573, 1584]
need_cash_df = pd.DataFrame([index, need_cash_ls], index=["index", "need_cashflow_month"]).T
need_cash_df['need_cashflow_annual'] = need_cash_df['need_cashflow_month']*12
need_cash_df['dividend_rate'] = 0.0335
need_cash_df['tax_rate'] = 0.154
need_cash_df['need_kapital'] = (need_cash_df['need_cashflow_annual']/(need_cash_df['dividend_rate']*(1-need_cash_df['tax_rate']))).astype(int)

In [39]:
## Interpolation
## naq => QQQ 보간
dataset_imputed = imputation(dataset, '^IXIC', 'QQQ')
dataset_imputed = imputation(dataset_imputed, '^GSPC', 'SPY')
dataset_imputed = imputation(dataset_imputed, '^GSPC', 'SCHD')

,index,need_cashflow_month,need_cashflow_annual,dividend_rate,tax_rate,need_kapital
0,최저1인,134,1608,0.0335,0.154,56737
1,최저2인,221,2652,0.0335,0.154,93574
2,최저3인,283,3396,0.0335,0.154,119826
3,최저4인,344,4128,0.0335,0.154,145654
4,중위1인,224,2688,0.0335,0.154,94844
5,중위2인,368,4416,0.0335,0.154,155816
6,중위3인,471,5652,0.0335,0.154,199428
7,중위4인,573,6876,0.0335,0.154,242616
8,상위,1584,19008,0.0335,0.154,670689


##### EDA

###### Type1

In [368]:
## 데이터 설정값
ticker_A = 'SCHD'
ticker_B = 'QQQ'

In [386]:
df = dataset_imputed[[ticker_A, ticker_B]].dropna()
df = df/df.iloc[0]

## data-split
df_ls = split_time_series(df, 5*252)

## calculate Rolling Returns
return_ls = []
ddr_ls = []
_idx = []
for _df in df_ls:
    _df = simulate_portfolio_assets(_df.pct_change().fillna(0), [(10,0), (9, 1), (8, 2), (7, 3), (6, 4), (5, 5), (4, 6), (3, 7), (2, 8), (1, 9), (0,10)])
    _df.columns = [ticker_A, '9:1', '8:2', '7:3', '6:4', '5:5', '4:6', '3:7', '2:8', '1:9', ticker_B]
    return_ls.append([anal_funcs.cal_YRR(_df, col, method ='g', unit = 'daily') for col in _df.columns])
    ddr_ls.append([anal_funcs.cal_drawdown_risk(_df, col) for col in _df.columns])
    _idx.append(_df.index[0])  
    
return_df = pd.DataFrame(return_ls, columns = _df.columns, index=_idx)
ddr_df = pd.DataFrame(ddr_ls, columns = _df.columns, index=_idx)

sort_df_type1 = pd.DataFrame([return_df.mean(), ddr_df.mean()], index=['Return', 'Volatility']).T

###### Type2

In [391]:
## 데이터 설정값
ticker_A = 'QQQ'
ticker_B = 'TLT'

In [395]:
df = dataset_imputed[[ticker_A, ticker_B]].dropna()
df = df/df.iloc[0]

## data-split
df_ls = split_time_series(df, 5*252)

## calculate Rolling Returns
return_ls = []
ddr_ls = []
_idx = []
for _df in df_ls:
    _df = simulate_portfolio_assets(_df.pct_change().fillna(0), [(10,0), (9, 1), (8, 2), (7, 3), (6, 4), (5, 5), (4, 6), (3, 7), (2, 8), (1, 9), (0,10)])
    _df.columns = [ticker_A, '9:1', '8:2', '7:3', '6:4', '5:5', '4:6', '3:7', '2:8', '1:9', ticker_B]
    return_ls.append([anal_funcs.cal_YRR(_df, col, method ='g', unit = 'daily') for col in _df.columns])
    ddr_ls.append([anal_funcs.cal_drawdown_risk(_df, col) for col in _df.columns])
    _idx.append(_df.index[0])  
    

return_df = pd.DataFrame(return_ls, columns = _df.columns, index=_idx)
ddr_df = pd.DataFrame(ddr_ls, columns = _df.columns, index=_idx)

sort_df_type2 = pd.DataFrame([return_df.mean(), ddr_df.mean()], index=['Return', 'Volatility']).T

###### Type3

In [422]:
## 데이터 설정값
ticker_A = 'QQQ'
ticker_B = 'SHY'

In [ ]:
df = dataset_imputed[[ticker_A, ticker_B]].dropna()
df = df/df.iloc[0]

## data-split
df_ls = split_time_series(df, 5*252)

## calculate Rolling Returns
return_ls = []
ddr_ls = []
_idx = []
for _df in df_ls:
    _df = simulate_portfolio_assets(_df.pct_change().fillna(0), [(10,0), (9, 1), (8, 2), (7, 3), (6, 4), (5, 5), (4, 6), (3, 7), (2, 8), (1, 9), (0,10)])
    _df.columns = [ticker_A, '9:1', '8:2', '7:3', '6:4', '5:5', '4:6', '3:7', '2:8', '1:9', ticker_B]
    return_ls.append([anal_funcs.cal_YRR(_df, col, method ='g', unit = 'daily') for col in _df.columns])
    ddr_ls.append([anal_funcs.cal_drawdown_risk(_df, col) for col in _df.columns])
    _idx.append(_df.index[0])  
    

return_df = pd.DataFrame(return_ls, columns = _df.columns, index=_idx)
ddr_df = pd.DataFrame(ddr_ls, columns = _df.columns, index=_idx)

sort_df_type2 = pd.DataFrame([return_df.mean(), ddr_df.mean()], index=['Return', 'Volatility']).T

###### Type4

In [ ]:
## 데이터 설정값
ticker_A = 'QQQ'
ticker_B = 'SHY'

In [ ]:
df = dataset_imputed[[ticker_A, ticker_B]].dropna()
df = df/df.iloc[0]

## data-split
df_ls = split_time_series(df, 5*252)

## calculate Rolling Returns
return_ls = []
ddr_ls = []
_idx = []
for _df in df_ls:
    _df = simulate_portfolio_assets(_df.pct_change().fillna(0), [(10,0), (9, 1), (8, 2), (7, 3), (6, 4), (5, 5), (4, 6), (3, 7), (2, 8), (1, 9), (0,10)])
    _df.columns = [ticker_A, '9:1', '8:2', '7:3', '6:4', '5:5', '4:6', '3:7', '2:8', '1:9', ticker_B]
    return_ls.append([anal_funcs.cal_YRR(_df, col, method ='g', unit = 'daily') for col in _df.columns])
    ddr_ls.append([anal_funcs.cal_drawdown_risk(_df, col) for col in _df.columns])
    _idx.append(_df.index[0])  
    

return_df = pd.DataFrame(return_ls, columns = _df.columns, index=_idx)
ddr_df = pd.DataFrame(ddr_ls, columns = _df.columns, index=_idx)

sort_df_type3 = pd.DataFrame([return_df.mean(), ddr_df.mean()], index=['Return', 'Volatility']).T

###### Type4

In [ ]:
## 데이터 설정값
ticker_A = 'QQQ'
ticker_B = 'GOLD'

In [ ]:
df = dataset_imputed[[ticker_A, ticker_B]].dropna()
df = df/df.iloc[0]

## data-split
df_ls = split_time_series(df, 5*252)

## calculate Rolling Returns
return_ls = []
ddr_ls = []
_idx = []
for _df in df_ls:
    _df = simulate_portfolio_assets(_df.pct_change().fillna(0), [(10,0), (9, 1), (8, 2), (7, 3), (6, 4), (5, 5), (4, 6), (3, 7), (2, 8), (1, 9), (0,10)])
    _df.columns = [ticker_A, '9:1', '8:2', '7:3', '6:4', '5:5', '4:6', '3:7', '2:8', '1:9', ticker_B]
    return_ls.append([anal_funcs.cal_YRR(_df, col, method ='g', unit = 'daily') for col in _df.columns])
    ddr_ls.append([anal_funcs.cal_drawdown_risk(_df, col) for col in _df.columns])
    _idx.append(_df.index[0])  
    

return_df = pd.DataFrame(return_ls, columns = _df.columns, index=_idx)
ddr_df = pd.DataFrame(ddr_ls, columns = _df.columns, index=_idx)

sort_df_type4 = pd.DataFrame([return_df.mean(), ddr_df.mean()], index=['Return', 'Volatility']).T

###### Type5

In [ ]:
## 데이터 설정값
ticker_A = 'QQQ'
ticker_B = 'CASH'

In [ ]:
df = dataset_imputed[[ticker_A, ticker_B]].dropna()
df = df/df.iloc[0]

## data-split
df_ls = split_time_series(df, 5*252)

## calculate Rolling Returns
return_ls = []
ddr_ls = []
_idx = []
for _df in df_ls:
    _df = simulate_portfolio_assets(_df.pct_change().fillna(0), [(10,0), (9, 1), (8, 2), (7, 3), (6, 4), (5, 5), (4, 6), (3, 7), (2, 8), (1, 9), (0,10)])
    _df.columns = [ticker_A, '9:1', '8:2', '7:3', '6:4', '5:5', '4:6', '3:7', '2:8', '1:9', ticker_B]
    return_ls.append([anal_funcs.cal_YRR(_df, col, method ='g', unit = 'daily') for col in _df.columns])
    ddr_ls.append([anal_funcs.cal_drawdown_risk(_df, col) for col in _df.columns])
    _idx.append(_df.index[0])  
    

return_df = pd.DataFrame(return_ls, columns = _df.columns, index=_idx)
ddr_df = pd.DataFrame(ddr_ls, columns = _df.columns, index=_idx)

sort_df_type4 = pd.DataFrame([return_df.mean(), ddr_df.mean()], index=['Return', 'Volatility']).T

###### Type6

In [ ]:
## 데이터 설정값
ticker_A = 'SCHD'
ticker_B = 'TLT'

In [ ]:
df = dataset_imputed[[ticker_A, ticker_B]].dropna()
df = df/df.iloc[0]

## data-split
df_ls = split_time_series(df, 5*252)

## calculate Rolling Returns
return_ls = []
ddr_ls = []
_idx = []
for _df in df_ls:
    _df = simulate_portfolio_assets(_df.pct_change().fillna(0), [(10,0), (9, 1), (8, 2), (7, 3), (6, 4), (5, 5), (4, 6), (3, 7), (2, 8), (1, 9), (0,10)])
    _df.columns = [ticker_A, '9:1', '8:2', '7:3', '6:4', '5:5', '4:6', '3:7', '2:8', '1:9', ticker_B]
    return_ls.append([anal_funcs.cal_YRR(_df, col, method ='g', unit = 'daily') for col in _df.columns])
    ddr_ls.append([anal_funcs.cal_drawdown_risk(_df, col) for col in _df.columns])
    _idx.append(_df.index[0])  
    

return_df = pd.DataFrame(return_ls, columns = _df.columns, index=_idx)
ddr_df = pd.DataFrame(ddr_ls, columns = _df.columns, index=_idx)

sort_df_type2 = pd.DataFrame([return_df.mean(), ddr_df.mean()], index=['Return', 'Volatility']).T

###### Type7

In [ ]:
## 데이터 설정값
ticker_A = 'SCHD'
ticker_B = 'SHY'

In [ ]:
df = dataset_imputed[[ticker_A, ticker_B]].dropna()
df = df/df.iloc[0]

## data-split
df_ls = split_time_series(df, 5*252)

## calculate Rolling Returns
return_ls = []
ddr_ls = []
_idx = []
for _df in df_ls:
    _df = simulate_portfolio_assets(_df.pct_change().fillna(0), [(10,0), (9, 1), (8, 2), (7, 3), (6, 4), (5, 5), (4, 6), (3, 7), (2, 8), (1, 9), (0,10)])
    _df.columns = [ticker_A, '9:1', '8:2', '7:3', '6:4', '5:5', '4:6', '3:7', '2:8', '1:9', ticker_B]
    return_ls.append([anal_funcs.cal_YRR(_df, col, method ='g', unit = 'daily') for col in _df.columns])
    ddr_ls.append([anal_funcs.cal_drawdown_risk(_df, col) for col in _df.columns])
    _idx.append(_df.index[0])  
    

return_df = pd.DataFrame(return_ls, columns = _df.columns, index=_idx)
ddr_df = pd.DataFrame(ddr_ls, columns = _df.columns, index=_idx)

sort_df_type2 = pd.DataFrame([return_df.mean(), ddr_df.mean()], index=['Return', 'Volatility']).T

###### Type8

In [ ]:
## 데이터 설정값
ticker_A = 'SCHD'
ticker_B = 'GOLD'

In [ ]:
df = dataset_imputed[[ticker_A, ticker_B]].dropna()
df = df/df.iloc[0]

## data-split
df_ls = split_time_series(df, 5*252)

## calculate Rolling Returns
return_ls = []
ddr_ls = []
_idx = []
for _df in df_ls:
    _df = simulate_portfolio_assets(_df.pct_change().fillna(0), [(10,0), (9, 1), (8, 2), (7, 3), (6, 4), (5, 5), (4, 6), (3, 7), (2, 8), (1, 9), (0,10)])
    _df.columns = [ticker_A, '9:1', '8:2', '7:3', '6:4', '5:5', '4:6', '3:7', '2:8', '1:9', ticker_B]
    return_ls.append([anal_funcs.cal_YRR(_df, col, method ='g', unit = 'daily') for col in _df.columns])
    ddr_ls.append([anal_funcs.cal_drawdown_risk(_df, col) for col in _df.columns])
    _idx.append(_df.index[0])  
    

return_df = pd.DataFrame(return_ls, columns = _df.columns, index=_idx)
ddr_df = pd.DataFrame(ddr_ls, columns = _df.columns, index=_idx)

sort_df_type2 = pd.DataFrame([return_df.mean(), ddr_df.mean()], index=['Return', 'Volatility']).T

###### Type9

In [ ]:
## 데이터 설정값
ticker_A = 'SCHD'
ticker_B = 'CASH'

In [ ]:
df = dataset_imputed[[ticker_A, ticker_B]].dropna()
df = df/df.iloc[0]

## data-split
df_ls = split_time_series(df, 5*252)

## calculate Rolling Returns
return_ls = []
ddr_ls = []
_idx = []
for _df in df_ls:
    _df = simulate_portfolio_assets(_df.pct_change().fillna(0), [(10,0), (9, 1), (8, 2), (7, 3), (6, 4), (5, 5), (4, 6), (3, 7), (2, 8), (1, 9), (0,10)])
    _df.columns = [ticker_A, '9:1', '8:2', '7:3', '6:4', '5:5', '4:6', '3:7', '2:8', '1:9', ticker_B]
    return_ls.append([anal_funcs.cal_YRR(_df, col, method ='g', unit = 'daily') for col in _df.columns])
    ddr_ls.append([anal_funcs.cal_drawdown_risk(_df, col) for col in _df.columns])
    _idx.append(_df.index[0])  
    

return_df = pd.DataFrame(return_ls, columns = _df.columns, index=_idx)
ddr_df = pd.DataFrame(ddr_ls, columns = _df.columns, index=_idx)

sort_df_type2 = pd.DataFrame([return_df.mean(), ddr_df.mean()], index=['Return', 'Volatility']).T

###### Type10

In [420]:
# plot_df = sort_df_type2.copy()

# # 색상 및 크기 설정
# colors = [color_norm, color_sub, color_sub, color_sub, color_sub, color_sub, color_sub, color_sub, color_sub, color_sub, color_cont]
# sizes = 500

# # 그래프 설정
# fig, ax = plt.subplots(figsize=figsize)

# # 점 그리기
# x = plot_df['Volatility']
# y = plot_df['Return']
# scatter = ax.scatter(x, y, c=colors, s=sizes, edgecolors="white", linewidth=2, alpha=0.9)


# # 데이터 레이블 추가
# labels = list(plot_df.index)
# for i, label in enumerate(labels):
#     plt.text(
#         x[i], y[i]-np.abs(y.max()-y.min())*0.05, label, fontsize=12, ha="center", va="center", 
#         color="white", fontweight="bold", bbox=dict(facecolor=colors[i], edgecolor='none', alpha=0.8, boxstyle="round,pad=0.3")
#     )

# # % 단위를 추가하는 포맷터 함수 정의
# def percent_formatter(x, pos):
#     return f"{int(x)}%"

# # X축, Y축에 % 포맷터 적용
# plt.gca().xaxis.set_major_formatter(FuncFormatter(percent_formatter))
# plt.gca().yaxis.set_major_formatter(FuncFormatter(percent_formatter))
# # 눈금과 축 간격 줄이기
# plt.gca().tick_params(axis="x", pad=1)  # X축 눈금 패딩 조정
# plt.gca().tick_params(axis="y", pad=1)  # Y축 눈금 패딩 조정

# # 축 설정
# plt.title(f"Return-Risk Profile : {df.index[0].year}~{df.index[-1].year}", fontsize=22, fontweight="bold", color=color_basic)
# plt.xlabel("Risk", fontsize=16, labelpad=0, fontweight="bold", color=color_around, loc="right")
# plt.ylabel("Return", fontsize=16, labelpad=-50, fontweight="bold", color=color_around, loc="top", rotation=0)
# plt.xticks(fontsize=10, color=color_around)
# plt.yticks(fontsize=10, color=color_around)
# # plt.xlim(2.9, 5.7)
# # plt.ylim(2.4, 6.9)
# plt.grid(color=color_around2, linestyle="--", linewidth=0.7, alpha=0.7)

# # 저장 및 출력
# plt.tight_layout()
# plt.show()

In [93]:
ddr_df.mean()

QQQ    3.468638
9:1    3.016826
8:2    2.583031
7:3    2.280050
6:4    2.038789
5:5    1.894638
4:6    1.812192
3:7    1.770501
2:8    1.839377
1:9    1.975473
TLT    2.211255
dtype: float64